In [1]:
import pandas as pd
from ddr import *
pd.options.mode.copy_on_write = True

In [2]:
# kolommen controleren controleren met df.columns zonder 'usecols='
vl = pd.read_csv('vrlijst_lijstdef.csv',
                  usecols=[0, 1, 2, 5, 8, 12, 16, 19],
                  dtype={'VERVALLEN': 'boolean', 'DATUM': 'string'})

In [13]:
vl.loc[vl['SOORTID'] == 'C']

,LIJSTID,CATEGORIE,LIJSTNAAM,DATUM,VERVALLEN,KOPPID,VRLIJSTTYPEID,SOORTID
128,CS00058881,ENDSEG,Entero conclusie,2021-11-08 00:00:00,True,CS00014119,NaN,C
172,CS00055070,REV,Huidig niveau van functioneren,2020-04-07 00:00:00,False,CS00012846,NaN,C
173,CS00055188,REV,Sensibiliteit,2020-04-29 00:00:00,False,CS00012883,NaN,C
174,CS00055186,REV,Motoriek,2020-04-29 00:00:00,False,CS00012881,NaN,C
175,CS00055280,AN,Cyclus,2020-05-20 00:00:00,False,CS00012911,NaN,C
...,...,...,...,...,...,...,...,...
54110,CS00062083,RTHARTS,Cardiac risk factors,2023-03-08 00:00:00,False,CS00015000,NaN,C
57977,CS00063251,ENDSEG,Aanlevering registraties Algemeen,2023-11-17 00:00:00,False,CS00015234,NaN,C
57978,CS00063250,ENDSEG,Aanlevering registraties Entero,2023-11-17 00:00:00,False,CS00015233,NaN,C
57992,CS00063247,ENDSEG,Aanlevering registraties ERCP,2023-11-17 00:00:00,False,CS00015230,NaN,C


In [4]:
vr = pd.read_csv('vrlijst_vragen.csv', low_memory=False,
                usecols=[0, 1, 3, 6, 7, 8, 12, 27, 33, 39, 40],
                dtype={'MAGLEEG': 'boolean', 'VERVALLEN': 'boolean', 'ENKELV': 'boolean',
                       'CONTROLID': 'Int64'})

In [5]:
vc = pd.read_csv('vrlijst_controls.csv',
                usecols=[0, 1, 4])

In [ ]:
def vraag_type(vraag):
    descr = vc.loc[vc['CONTROLID'] == vraag.CONTROLID].to_dict('records')
    return descr[0]['NAAM']

In [6]:
tl = pd.read_csv('vrlijst_treelay.csv',
                 usecols=[0, 1, 2, 3, 4, 8, 9, 12, 13],
                 dtype={'VOLGNR':'Int64', 'HERHAALBAA': 'boolean', 'ONZZANTW': 'boolean', 'NEWPAGE': 'boolean'})

In [7]:
start = Lijst.make_one(vl.loc[vl['LIJSTID'] == 'CS00065113'])

In [9]:
def add_children(vlnode, distance=0):
    tlnodes = tl.loc[tl['LIJSTID'] == vlnode.LIJSTID].copy()
    tlnodes.sort_values(by='VOLGNR', inplace=True)
    for node in tlnodes.to_dict('records'):
        childid = node['CHILDID']
        indent = '  ' * distance
        onz = node['ONZZANTW']
        if node['TYPE'] == 'V':
            vraag = Vraag.make_one(vr.loc[vr['VRAAGID'] == childid])
            vlnode.children.append(vraag)
            print(f"{indent}{node['VOLGNR']} V: {vraag} type={vraag_type(vraag)} verberg={onz}")
        elif node['TYPE'] == 'L':
            lijst = Lijst.make_one(vl.loc[vl['LIJSTID'] == childid])
            add_children(lijst, distance+1)
            vlnode.children.append(lijst)
            print(f"{indent}{node['VOLGNR']} L: {lijst} verberg={onz}")
        else:
            print(f"{indent}onbekend type {node['TYPE']}")

In [10]:
add_children(start)

0 V: Reden van aanmelding MDO (CS00276754) vervallen=False type=RTF Invoer Memo verberg=False
1 V: Datum van aanmelding (CS00276755) vervallen=False type=Datum verberg=False
2 V: Opmerkingen bij MDO (CS00276756) vervallen=False type=Memo verberg=False
3 V: Patiënt eerder besproken (CS00214848) vervallen=False type=Keuzeselectiegroep verberg=True
